In [ ]:
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os

from pykern.pkcollections import PKDict
from pykern import pkdebug
import rslaser.rspulse.pulse as rsp
import rslaser.rspulse.gauss_hermite as rsgh
import rslaser.utils.plot_fields as rspf
import rslaser.utils.constants as rsc

import scipy.constants as const

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

In [ ]:
# specify all top-level input parameters

# central values
a0 = 0.01             # 0.85e-9 * lambda [microns] * Sqrt(Intensity [W/cm^2])
lambda0 = 8.e-7       # wavelength [m]
tau_fwhm = 1.e-10     # FWHM pulse length [s]
w0 = 400.e-6          # waist size [m]
_k0 = rsc.TWO_PI/lambda0    # wavenumber [radians/m]   DERIVED
_zR = 0.5*_k0*w0*2          # Rayleigh range [m]       DERIVED
z_waist = 0.0         # longitudinal waist position [m]
z_center = 0.0        # longitudinal position of pulse center [m]

# deviations
d_lambda = 0.         # full wavelength chirp across laser pulse [m]
dw0x = 0.0            # deviation of horizontal waist from w0
dw0y = 0.0            # deviation of vertical waist from w0
dzwx = 0.0            # deviation of horizontal waist position from z_waist
dzwy = 0.0            # deviation of vertical waist position from z_waist
x_shift = 0.          # bulk horizontal shift of pulse position
y_shift = 0.          # bulk vertical shift of pulse position

# numerical values
num_slices = 3        # desired number of slices (i.e. SRW wavefronts) to represent the pulse

# other DERIVED quantities
d_to_w = z_waist - z_center   # so-called 'drift to waist' (required by SRW)

In [ ]:
# create a dictionary of all relevant physical and numerical quantities
data_in=PKDict()

data_in.a0 = a0
data_in.lambda0 = lambda0
data_in.tau_fwhm = tau_fwhm
data_in.w0 = w0
data_in.z_waist = z_waist
data_in.z_center = z_center
data_in.d_to_w = z_waist - z_center

data_in.d_lambda = d_lambda
data_in.dw0x = dw0x
data_in.dw0y = dw0y
data_in.dzwx = dzwx
data_in.dzwy = dzwy
data_in.x_shift = x_shift
data_in.y_shift = y_shift

data_in.nslice = num_slices

#create laser pulse
LP = rsp.LaserPulse(data_in)

In [ ]:
# specify the desired plots
plot_2d_zy_explicit = True
plot_1d_z_explicit = True
plot_1d_z_envelope = True

In [ ]:
# plot fields in a z-y plane that intersects the x=0 axis
# we are assuming linear polarization along the x-axis

# Turn on print statements for a particular file, a specified module, or *all* rslaser modules
#pkdebug.init(control='gauss_hermite.py')
#pkdebug.init(control='rspulse')
#pkdebug.init(control='rslaser')

if plot_2d_zy_explicit:
    
    # longitudinal resolution
    hres = 64
    dh = LP.envelope.lambda0 / hres

    # specify the horizontal min's and max's
    zyMinH = -8. * LP.envelope.lambda0
    zyMaxH =  4. * LP.envelope.lambda0

    # specify the number of horizontal mesh points
    zyNumH = int((zyMaxH - zyMinH) / dh)
    
    # vertical resolution
    vres = 20
    dv = LP.envelope.w0 / vres

    # specify the vertical min's and max's
    zyMaxV =  1.5 * LP.envelope.w0
    zyMinV = -zyMaxV

    # specify the number of vertical mesh points
    zyNumV = int((zyMaxV - zyMinV) / dv)

    # create the horiontal (z) and vertical (y) arrays
    zArr = np.zeros(zyNumH)
    yArr = np.zeros(zyNumV)

    for iLoop in range(zyNumH):
        zArr[iLoop] = zyMinH + iLoop * (zyMaxH-zyMinH) / (zyNumH-1)

    for jLoop in range(zyNumV):
        yArr[jLoop] = zyMinV + jLoop * (zyMaxV-zyMinV) / (zyNumV-1)

    # Choose values of x,t for plot
    xValue = 0.
    tValue = 0.

    with plt.style.context(('seaborn-poster')):
        fig = plt.figure()
        ax = fig.gca()
        rspf.plot_2d_zy(zArr, yArr, LP.envelope, ax, _x=xValue, _t=tValue, _time_explicit=True, _nlevels=100)

In [ ]:
# plot a lineout of Ex along z
# we are assuming linear polarization along the x-axis
# the time-dependent oscillations will be included

# Turn on print statements for a particular file, a specified module, or *all* rslaser modules
#pkdebug.init(control='gauss_hermite.py')
#pkdebug.init(control='rspulse')
#pkdebug.init(control='rslaser')

if plot_1d_z_explicit:
    z_max = 40. * LP.envelope.lambda0
    z_min = 0.

    # resolution
    z_res = 32
    del_z = lambda0 / z_res
    num_pts = int((z_max - z_min) / del_z)
    print('\n  num_pts = ', num_pts)

    z_vals = np.zeros(num_pts)
    for ind in range(num_pts):
        z_vals[ind] = z_min + ind*del_z
    
    x_val = 0.
    y_val = 0.
    t_val = 0.
    time_explicit = True

    with plt.style.context(('seaborn-poster')):
        fig = plt.figure()
        ax = fig.gca()
        rspf.plot_1d_z(z_vals, LP.envelope, ax, x_val, y_val, t_val, time_explicit)

    plt.tight_layout()

In [ ]:
# plot a lineout of Ex along z
# we are assuming linear polarization along the x-axis
# this time, we plot only the envelope

# Turn on print statements for a particular file, a specified module, or *all* rslaser modules
#pkdebug.init(control='gauss_hermite.py')
#pkdebug.init(control='rspulse')
#pkdebug.init(control='rslaser')

if plot_1d_z_envelope:
    z_max = 3. * LP.envelope.tau_fwhm * const.c
    z_min = -z_max

    # resolution
    z_res = 4
    del_z = LP.envelope.w0 / z_res
    num_pts = int((z_max - z_min) / del_z)
    # print('\n  num_pts = ', num_pts)

    z_vals = np.zeros(num_pts)
    for ind in range(num_pts):
        z_vals[ind] = z_min + ind*del_z
    
    x_val = 0.
    y_val = 0.
    t_val = 0.
    time_explicit = False

    with plt.style.context(('seaborn-poster')):
        fig = plt.figure()
        ax = fig.gca()
        rspf.plot_1d_z(z_vals, LP.envelope, ax, x_val, y_val, t_val, time_explicit)

    plt.tight_layout()